In [1]:
import boto3
import pandas as pd
import os

In [2]:
AWS_S3_BUCKET_NAME = 'border-crossing-bucket'
AWS_REGION = 'us-east-1'
AWS_ACCESS_KEY = os.environ.get('AWS_ACCESS_KEY')
AWS_SECRET_KEY = os.environ.get('AWS_SECRET_KEY')

In [ ]:
!wget -O border_crossings.csv https://data.transportation.gov/api/views/keg4-3bc2/rows.csv?accessType=DOWNLOAD

In [5]:
border_crossing_csv = 'border_crossings.csv'

In [6]:
s3_file_name = 'border_crossings_s3.csv'

In [8]:
s3_client = boto3.client(
        service_name='s3',
        region_name=AWS_REGION,
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY
    )

response = s3_client.upload_file(border_crossing_csv, AWS_S3_BUCKET_NAME, s3_file_name)
print(response)

None
